# Handling Volume with Apache Spark for Feature Extraction

Use Apache Spark to perform feature extraction from preprocessed images.

## License

MIT License

Copyright (c) 2019 PT Bukalapak.com

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Software Version

In [1]:
import sys
print("Python %s" % sys.version)
import base64
import pickle
import shutil

Python 3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 21:52:21) 
[GCC 7.3.0]


In [2]:
import numpy as np
print("NumPy %s" % np.__version__)

NumPy 1.15.4


In [3]:
import tensorflow as tf
print("TensorFlow %s" % tf.__version__)

TensorFlow 1.14.0


In [4]:
import pyspark
print("PySpark %s" % pyspark.__version__)
from pyspark.sql import SparkSession, Row

PySpark 2.3.3


## Perform Feature Extraction using Notebook (NB)

### Settings

In [5]:
print("Built With CUDA:", tf.test.is_built_with_cuda())

Built With CUDA: False


In [6]:
print("Keras FP Precision:", tf.keras.backend.floatx())

Keras FP Precision: float32


In [7]:
LOCAL_PROJECT_URL = '/home/jovyan/work/'

### Create Spark Context

Set config to run Spark locally with 2 threads.

In [8]:
APP_NAME = "bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr"
spark = SparkSession \
    .builder \
    .appName(APP_NAME) \
    .config("spark.master", "local[2]") \
    .getOrCreate()

In [9]:
sc = spark.sparkContext

In [10]:
sc

<SparkContext master=local[2] appName=bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr>

In [11]:
sc.getConf().getAll()

[('spark.app.id', 'local-1565249163712'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', 'a5a5e8400117'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr'),
 ('spark.driver.port', '34403'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.master', 'local[2]')]

### Generate Preprocessed Images for Input

Generate preprocessed images and store them in HDFS Orc file. Preprocessed images normally come from previous process.

In [12]:
# ped: preprocessed image
def convert_image_url_to_ped(x):
    import io, base64, pickle
    from PIL import Image as pil_image
    import tensorflow as tf
    # Convert URL to PIL image
    image_url = x[0]
    image_pil = pil_image.open(image_url)
    # Make Sure the Image is in RGB (not BW)
    if image_pil.mode != 'RGB':
        image_pil = image_pil.convert('RGB')
    # Resize Image
    target_size = (224, 224)
    width_height_tuple = (target_size[1], target_size[0])
    if image_pil.size != width_height_tuple:
        image_pil = image_pil.resize(width_height_tuple, pil_image.NEAREST)
    # Normalise Image
    image_np = tf.keras.preprocessing.image.img_to_array(image_pil)
    image_np = tf.keras.applications.vgg16.preprocess_input(image_np)
    # Convert numpy array to string
    image_ped = base64.b64encode(pickle.dumps(image_np)).decode('UTF-8')
    
    return ["local_disk", image_url, "jeket", image_ped]

In [13]:
def convert_image_url_to_ped_orc(image_url_pathfilename, image_ped_orc_pathfilename):
    image_url_file_rdd = sc.textFile(image_url_pathfilename)
    print("        Number of Partitions:", image_url_file_rdd.getNumPartitions())
    image_url_list_rdd = image_url_file_rdd.map(lambda x: x.split('\n'))
    image_ped_list_rdd = image_url_list_rdd.map(lambda x: convert_image_url_to_ped(x))
    image_ped_dict_rdd = image_ped_list_rdd.map(lambda x: Row(tid=x[0], 
                                                              iid=x[1], 
                                                              l=x[2], 
                                                              i_ped=x[3]))
    image_ped_dict_df = spark.createDataFrame(image_ped_dict_rdd)
    image_ped_dict_df.write.save(image_ped_orc_pathfilename, format="orc")

In [14]:
# Input file path
image_url_pathfilename = "file:{0}data/image_paths_10.txt".format(LOCAL_PROJECT_URL)
# Output file path
image_ped_orc_directory = "{0}data/images_ped.orc".format(LOCAL_PROJECT_URL)
image_ped_orc_pathfilename = "file:%s" % image_ped_orc_directory
# Remove existing output directory
shutil.rmtree(image_ped_orc_directory, ignore_errors=True)
# Start preprocessing
convert_image_url_to_ped_orc(image_url_pathfilename, image_ped_orc_pathfilename)

        Number of Partitions: 2


### Extract Features

Extract features from Medusa model based on VGG16 model[1] pretrained with imagenet.

In [15]:
# input is the list of [tid, iid, l, i_ped] from preprocessed image table
# output is the list of [tid, iid, fc1, fc2, pred] for features table
# where:
#  - tid: table ID
#  - iid: image ID
#  - l: label
#  - i_ped: preprocessed image
#  - fc1: output of fully connected layer 1
#  - fc2: output of fully connected layer 2
#  - pred: output of prediction layer (the inference result)

def extract_features(xs):
    import base64, pickle
    import numpy as np
    import tensorflow as tf
    # Extract input lists
    image_info_lists = []
    images_array = []
    for x in xs:
        image_info_lists.append([x.tid, x.iid])
        images_array.append(pickle.loads(base64.b64decode(x.i_ped.encode('UTF-8'))))
    images_np = np.array(images_array)
    # Load VGG16 model
    vgg = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
    # Construct medusa model
    vgg_input = vgg.input
    vgg_fc1_output = vgg.get_layer('fc1').output
    vgg_fc2_output = vgg.get_layer('fc2').output
    vgg_predictions_output = vgg.get_layer('predictions').output
    medusa = tf.keras.models.Model(inputs=vgg_input, 
                                   outputs=[vgg_fc1_output, 
                                            vgg_fc2_output, 
                                            vgg_predictions_output])
    # Extract features
    features = medusa.predict(images_np)
    # Add features to image table lists
    if len(image_info_lists) != len(features[0]):
        raise ValueError('The total of the image table lists is not ' +
                         'the same as the total of Medusa feature lists')
    for i in range(len(image_info_lists)):
        # append fc1
        image_info_lists[i].append(
            base64.b64encode(pickle.dumps(features[0][i])).decode('UTF-8')
        )
        # append fc2
        image_info_lists[i].append(
            base64.b64encode(pickle.dumps(features[1][i])).decode('UTF-8')
        )
        # append prediction
        image_info_lists[i].append(
            base64.b64encode(pickle.dumps(features[2][i])).decode('UTF-8')
        )

    return iter(image_info_lists)

In [16]:
def convert_image_ped_orc_to_infr_orc(image_ped_orc_pathfilename,
                                      image_infr_orc_pathfilename):
    image_ped_dict_df = spark.read.orc(image_ped_orc_pathfilename)
    image_ped_dict_rdd = image_ped_dict_df.rdd
    print("        Number of Partitions:", image_ped_dict_rdd.getNumPartitions())
    image_infr_list_rdd = image_ped_dict_rdd.mapPartitions(extract_features)
    image_infr_dict_rdd = image_infr_list_rdd.map(lambda x: Row(tid=x[0],
                                                                iid=x[1],
                                                                fc1=x[2],
                                                                fc2=x[3],
                                                                pred=x[4]))
    image_infr_dict_df = spark.createDataFrame(image_infr_dict_rdd)
    image_infr_dict_df.write.save(image_infr_orc_pathfilename, format="orc")

In [17]:
# Input file path
image_ped_orc_directory = "{0}data/images_ped.orc".format(LOCAL_PROJECT_URL)
image_ped_orc_pathfilename = "file:{0}".format(image_ped_orc_directory)
# Output file path
image_infr_orc_directory = "{0}data/images_infr.orc".format(LOCAL_PROJECT_URL)
image_infr_orc_pathfilename = "file:{0}".format(image_infr_orc_directory)
# Remove existing output directory
shutil.rmtree(image_infr_orc_directory, ignore_errors=True)
# Start extracting features
convert_image_ped_orc_to_infr_orc(image_ped_orc_pathfilename, 
                                  image_infr_orc_pathfilename)

        Number of Partitions: 2


### Check Result

In [18]:
# Read features
image_infr_orc_directory = "{0}data/images_infr.orc".format(LOCAL_PROJECT_URL)
image_infr_orc_pathfilename = "file:{0}".format(image_infr_orc_directory)
image_infr_dict_df = spark.read.orc(image_infr_orc_pathfilename)

In [19]:
image_infr_dict_rdd = image_infr_dict_df.rdd
image_info_lists = image_infr_dict_rdd.take(1)[0]

In [20]:
image_info_lists.tid

'local_disk'

In [21]:
image_info_lists.iid

'./data/images/jacket copy 0.png'

In [22]:
image_info_lists.fc1[:100]

'gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShL'

In [23]:
image_info_lists.fc2[:100]

'gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShL'

In [24]:
image_info_lists.pred[:100]

'gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShL'

In [25]:
def convert_pbs_to_infr_tensor(pbs):
    return np.array(
        [list(pickle.loads(base64.b64decode(pbs.encode('UTF-8'))))])

In [26]:
tf.keras.applications.vgg16.decode_predictions(
    convert_pbs_to_infr_tensor(image_info_lists.pred))

[[('n04370456', 'sweatshirt', 0.971396),
  ('n04599235', 'wool', 0.015504221),
  ('n03595614', 'jersey', 0.005360415),
  ('n02963159', 'cardigan', 0.0013214782),
  ('n03594734', 'jean', 0.0011332377)]]

Expected result:
```
[[('n04370456', 'sweatshirt', 0.971396),
  ('n04599235', 'wool', 0.015504221),
  ('n03595614', 'jersey', 0.005360415),
  ('n02963159', 'cardigan', 0.0013214782),
  ('n03594734', 'jean', 0.0011332377)]]
```

### Close Spark Context

In [27]:
sc.stop()

In [28]:
spark.stop()

## Perform Feature Extraction using Spark Submit (SS)

Use previously generated [preprocessed images](#Generate-Preprocessed-Images-for-Input) for the input.

### Extract Features

In [29]:
%%writefile bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr.py
# Copyright (c) 2019 PT Bukalapak.com
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

from pyspark.sql import SparkSession


APP_NAME = "bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr"


def extract_features(xs):
    import base64, pickle
    import numpy as np
    import tensorflow as tf
    # Extract input lists
    image_info_lists = []
    images_array = []
    for x in xs:
        image_info_lists.append([x.tid, x.iid])
        images_array.append(pickle.loads(base64.b64decode(x.i_ped.encode('UTF-8'))))
    images_np = np.array(images_array)
    # Load VGG16 model
    vgg = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
    # Construct medusa model
    vgg_input = vgg.input
    vgg_fc1_output = vgg.get_layer('fc1').output
    vgg_fc2_output = vgg.get_layer('fc2').output
    vgg_predictions_output = vgg.get_layer('predictions').output
    medusa = tf.keras.models.Model(inputs=vgg_input, 
                                   outputs=[vgg_fc1_output, 
                                            vgg_fc2_output, 
                                            vgg_predictions_output])
    # Extract features
    features = medusa.predict(images_np)
    # Add features to image table lists
    if len(image_info_lists) != len(features[0]):
        raise ValueError('The total of the image table lists is not ' +
                         'the same as the total of Medusa feature lists')
    for i in range(len(image_info_lists)):
        # append fc1
        image_info_lists[i].append(
            base64.b64encode(pickle.dumps(features[0][i])).decode('UTF-8')
        )
        # append fc2
        image_info_lists[i].append(
            base64.b64encode(pickle.dumps(features[1][i])).decode('UTF-8')
        )
        # append prediction
        image_info_lists[i].append(
            base64.b64encode(pickle.dumps(features[2][i])).decode('UTF-8')
        )

    return iter(image_info_lists)


def main(spark):
    from pyspark.sql import Row
    # Input
    image_ped_orc_pathfilename = \
        "file:/home/jovyan/work/" + \
        "data/images_ped.orc"
    # Output
    image_infr_orc_pathfilename = \
        "file:/home/jovyan/work/" + \
        "data/images_infr_ss.orc"
    # Read input file
    image_ped_dict_df = spark.read.orc(image_ped_orc_pathfilename)
    image_ped_dict_rdd = image_ped_dict_df.rdd
    print("        Number of Partitions:", image_ped_dict_rdd.getNumPartitions())
    # Extract features
    image_infr_list_rdd = image_ped_dict_rdd.mapPartitions(extract_features)
    # Write output file
    image_infr_dict_rdd = image_infr_list_rdd.map(lambda x: Row(tid=x[0],
                                                                iid=x[1],
                                                                fc1=x[2],
                                                                fc2=x[3],
                                                                pred=x[4]))
    image_infr_dict_df = spark.createDataFrame(image_infr_dict_rdd)
    image_infr_dict_df.write.save(image_infr_orc_pathfilename, format="orc")


if __name__ == "__main__":
    # Configure Spark
    spark = SparkSession \
        .builder \
        .appName(APP_NAME) \
        .getOrCreate()
    main(spark)
    spark.stop()

Writing bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr.py


__Note__: Don't forget to delete existing `images_infr_ss.orc` directory in `data/`. Following Spark implementation does not overwrite existing data but it will throw error.

__Note__: Initial run will take very long as the Spark code needs to download about 500 MB of VGG16 model.

In [30]:
%%bash
/usr/local/spark/bin/spark-submit \
    --executor-memory 2g --executor-cores 1 --num-executors 2 \
    bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr.py

2019-08-08 07:28:43 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2019-08-08 07:28:44 INFO  SparkContext:54 - Running Spark version 2.3.3
2019-08-08 07:28:44 INFO  SparkContext:54 - Submitted application: bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr
2019-08-08 07:28:44 INFO  SecurityManager:54 - Changing view acls to: jovyan
2019-08-08 07:28:44 INFO  SecurityManager:54 - Changing modify acls to: jovyan
2019-08-08 07:28:44 INFO  SecurityManager:54 - Changing view acls groups to: 
2019-08-08 07:28:44 INFO  SecurityManager:54 - Changing modify acls groups to: 
2019-08-08 07:28:44 INFO  SecurityManager:54 - SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
2019-08-08 07:28:44 INFO  Utils:54 - Successfully started service 'sparkDri

W0808 07:28:49.296257 140305231288128 deprecation.py:506] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
2019-08-08 07:28:50.597186: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-08-08 07:28:50.619602: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2208000000 Hz
2019-08-08 07:28:50.620007: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x558fc9e2bef0 executing computations on platform Host. Devices:
2019-08-08 07:28:50.620027: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-08-08 07:28:50.652053:

### Check Result

__Settings__

In [31]:
LOCAL_PROJECT_URL = '/home/jovyan/work/'

__Create Spark Context__

Set config to run Spark locally with 2 threads.

In [32]:
APP_NAME = "bukalapak-core-ai.big-data-3v.volume-spark-ftr-extr"
spark = SparkSession \
    .builder \
    .appName(APP_NAME) \
    .config("spark.master", "local[2]") \
    .getOrCreate()

In [33]:
sc = spark.sparkContext

__Read Features__

In [34]:
image_infr_orc_directory = "{0}data/images_infr_ss.orc".format(LOCAL_PROJECT_URL)
image_infr_orc_pathfilename = "file:{0}".format(image_infr_orc_directory)
image_infr_dict_df = spark.read.orc(image_infr_orc_pathfilename)

In [35]:
image_infr_dict_rdd = image_infr_dict_df.rdd
image_info_lists = image_infr_dict_rdd.take(1)[0]

In [36]:
image_info_lists.tid

'local_disk'

In [37]:
image_info_lists.iid

'./data/images/jacket copy 0.png'

In [47]:
image_info_lists.fc1[:300]

'gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShLAU0AEIVxBmNudW1weQpkdHlwZQpxB1gCAAAAZjRxCEsASwGHcQlScQooSwNYAQAAADxxC05OTkr/////Sv////9LAHRxDGKJQgBAAAAAAAAAAAAAAEdVyEAAAAAAAAAAAAAAAAApiDRB3lXQQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGmbH0Fhm8M+AAAAAAAAAAAA'

In [48]:
image_info_lists.fc2[:300]

'gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShLAU0AEIVxBmNudW1weQpkdHlwZQpxB1gCAAAAZjRxCEsASwGHcQlScQooSwNYAQAAADxxC05OTkr/////Sv////9LAHRxDGKJQgBAAAAAAAAAAAAAAAAAAABHIBdAAAAAAAAAAAAAAAAAAAAAAAAAAACZhUpA+UWVPgAAAAAAAAAAvHmiPwAAAAAAAAAAZqfmP8myWkAA'

In [49]:
image_info_lists.pred[:300]

'gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEBSwCFcQJDAWJxA4dxBFJxBShLAU3oA4VxBmNudW1weQpkdHlwZQpxB1gCAAAAZjRxCEsASwGHcQlScQooSwNYAQAAADxxC05OTkr/////Sv////9LAHRxDGKJQqAPAACD9II0cjczMPWpKzJ0zekxmvgHMzXB/TIkyTUxdL+pMP67JjDn7zUwqVEuMHolbC8U8SIvAOIeMOmOOTGMbyAvF56MMJb8tTFN'

In [43]:
def convert_pbs_to_infr_tensor(pbs):
    return np.array(
        [list(pickle.loads(base64.b64decode(pbs.encode('UTF-8'))))])

In [44]:
tf.keras.applications.vgg16.decode_predictions(
    convert_pbs_to_infr_tensor(image_info_lists.pred))

[[('n04370456', 'sweatshirt', 0.971396),
  ('n04599235', 'wool', 0.015504221),
  ('n03595614', 'jersey', 0.005360415),
  ('n02963159', 'cardigan', 0.0013214782),
  ('n03594734', 'jean', 0.0011332377)]]

Expected result:
```
[[('n04370456', 'sweatshirt', 0.971396),
  ('n04599235', 'wool', 0.015504221),
  ('n03595614', 'jersey', 0.005360415),
  ('n02963159', 'cardigan', 0.0013214782),
  ('n03594734', 'jean', 0.0011332377)]]
```

__Close Spark Context__

In [45]:
sc.stop()

In [46]:
spark.stop()

## References

1. K Simonyan, A Zisserman. 2014. Very Deep Convolutional Networks for Large-Scale Image Recognition. CoRR, abs/1409.1556. http://arxiv.org/abs/1409.1556.